<a href="https://colab.research.google.com/github/RichardJPovinelli/Evolutionary_Computation_Course/blob/main/Interactive_Evolution_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Interactive Evolutionary Algorithm (IEA) for Abstract Images

This notebook implements a **simple interactive evolutionary algorithm** (IEA) where **you** act as the fitness function.

- Each individual is a small abstract image (a grid of colors).
- You rate the images based on how much you like them.
- The algorithm uses your ratings to evolve new generations.

**Instructions**

1. Run the code cell below.
2. You will see a grid of images and sliders for rating.
3. Rate each image from 0 to 5.
4. Click **"Evolve to Next Generation"**.
5. Watch how the population changes over generations.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import (
    VBox, HBox, Box, Button, ToggleButtons, Layout,
    HTML,
    Image as WImage
)

import io
from PIL import Image

from IPython.display import display, clear_output
from IPython import get_ipython

# Ensure an inline matplotlib backend so plt.show() does not block
ip = get_ipython()
if ip is not None:
    try:
        ip.run_line_magic("matplotlib", "inline")
    except Exception:
        pass

# =======================
# Configuration
# =======================

# Image / genome settings
IMG_HEIGHT = 8      # pixels
IMG_WIDTH  = 8      # pixels
CHANNELS   = 3      # RGB

# Display: target square size (px) for each image output
IMG_DISPLAY_SIZE_PX = 140

# Evolution settings
POP_SIZE       = 12      # number of individuals per generation
ELITE_COUNT    = 4       # how many top-rated individuals to keep
MUTATION_RATE  = 0.15
MUTATION_STD   = 0.25
MAX_GENERATIONS = 50     # not enforced, just for reference

# UI settings
MAX_RATING = 5           # (unused now) kept for compatibility


# =======================
# Representation & Operators
# =======================

class Individual:
    def __init__(self, genome=None):
        if genome is None:
            # Random RGB values in [0, 1], with some Gaussian structure
            genome = np.clip(
                np.random.randn(IMG_HEIGHT, IMG_WIDTH, CHANNELS) * 0.5 + 0.5,
                0, 1
            )
        self.genome = genome.astype(np.float32)
        self.fitness = 0.0

    def copy(self):
        clone = Individual(self.genome.copy())
        clone.fitness = self.fitness
        return clone


def mutate(genome, mutation_rate=MUTATION_RATE, mutation_std=MUTATION_STD):
    '''
    Gaussian mutation applied to a random subset of pixels.
    '''
    mask = np.random.rand(*genome.shape) < mutation_rate
    noise = np.random.randn(*genome.shape) * mutation_std
    mutated = genome.copy()
    mutated[mask] = mutated[mask] + noise[mask]
    mutated = np.clip(mutated, 0.0, 1.0)
    return mutated


def crossover(parent1, parent2):
    '''
    Simple uniform crossover at the pixel level.
    '''
    mask = np.random.rand(IMG_HEIGHT, IMG_WIDTH, CHANNELS) < 0.5
    child_genome = np.where(mask, parent1.genome, parent2.genome)
    return Individual(child_genome)


def init_population(size=POP_SIZE):
    return [Individual() for _ in range(size)]


def evolve_population(population, ratings):
    '''
    Evolve a new population given current population and user ratings.

    population: list[Individual]
    ratings:    list[int] in [0..2] (0=Do not like, 1=Meh, 2=Like)
    '''
    # Assign fitness
    for ind, r in zip(population, ratings):
        ind.fitness = float(r)

    # Sort by fitness descending
    population = sorted(population, key=lambda ind: ind.fitness, reverse=True)

    # Elitism: keep the best individuals
    elites = [ind.copy() for ind in population[:ELITE_COUNT]]

    # If all ratings are zero, just re-randomize the population
    if max(ratings) == 0:
        return init_population(len(population)), elites

    # Selection probabilities proportional to (nonnegative) fitness
    fitnesses = np.array([max(ind.fitness, 0.0) for ind in population], dtype=np.float32)
    if fitnesses.sum() == 0:
        probs = np.ones_like(fitnesses) / len(fitnesses)
    else:
        probs = fitnesses / fitnesses.sum()

    def select_parent():
        idx = np.random.choice(len(population), p=probs)
        return population[idx]

    # Generate offspring to refill population
    new_population = elites.copy()
    while len(new_population) < len(population):
        p1 = select_parent()
        p2 = select_parent()
        child = crossover(p1, p2)
        child.genome = mutate(child.genome)
        new_population.append(child)

    return new_population, elites


# =======================
# Visualization Helpers
# =======================


def genome_to_png_bytes(genome):
    """Convert a numpy RGB array in [0,1] to raw PNG bytes for widget.Image."""
    arr = (np.clip(genome, 0, 1) * 255).astype(np.uint8)
    with io.BytesIO() as buf:
        Image.fromarray(arr).save(buf, format='PNG')
        return buf.getvalue()


def plot_individual(individual, ax):
    # Kept for backward compat; rarely used now
    ax.imshow(individual.genome)
    ax.axis('off')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_frame_on(False)
    for spine in ax.spines.values():
        spine.set_visible(False)
    ax.set_position([0, 0, 1, 1])


# =======================
# Interactive EA Class
# =======================

class InteractiveEA:
    def __init__(self):
        self.population = init_population()
        self.generation = 0
        self.history_best = []

        # Widgets
        self.button_evolve = Button(
            description='Evolve to Next Generation',
            button_style='success',
            layout=Layout(width='250px')
        )
        self.button_randomize = Button(
            description='Randomize Population',
            button_style='warning',
            layout=Layout(width='200px')
        )
        self.info_label = HTML()
        self.rating_inputs = []

        # Wire up callbacks
        self.button_evolve.on_click(self.on_evolve_clicked)
        self.button_randomize.on_click(self.on_randomize_clicked)

        # Build rating widgets once; ToggleButtons expose a mutable options list
        options = [("👎", 0), ("🤷", 1), ("👍", 2)]
        horizontal_layout = Layout(
            width='100%',
            display='flex',
            flex_flow='row',
            justify_content='space-around'
        )
        self.rating_inputs = [
            ToggleButtons(
                value=1,
                options=options,
                description='',
                layout=horizontal_layout,
                style={'button_width': '42px'}
            )
            for _ in range(len(self.population))
        ]

        # Initial display
        self.update_display()

    def update_display(self):
        # Update instructions
        self.info_label.value = (
            f'<b>Generation:</b> {self.generation} '
            f'&nbsp;|&nbsp; Select one of the emojis under each image, then click <b>Evolve</b>.'
            f' &nbsp;&nbsp;Legend: 👎 = Do not like, 🤷 = Meh, 👍 = Like'
        )

        # Use a flex Box for responsive layout
        box_layout = Layout(
            display='flex',
            flex_flow='row wrap',
            justify_content='center',
            align_items='flex-start',
            width='100%',
            overflow_x='hidden',
            overflow_y='visible'
        )

        size_px = IMG_DISPLAY_SIZE_PX

        # Build image widgets so every individual always renders
        image_widgets = [
            WImage(
                value=genome_to_png_bytes(ind.genome),
                format='png',
                layout=Layout(width=f'{size_px}px', height=f'{size_px}px')
            )
            for ind in self.population
        ]

        # Pair each image with its rating control
        image_input_pairs = [
            VBox(
                [img_widget, input_widget],
                layout=Layout(
                    align_items='center',
                    margin='0px 6px 12px 6px',
                    width=f'{size_px}px',
                    min_width=f'{size_px}px',
                    max_width=f'{size_px}px',
                    flex=f'0 0 {size_px}px',
                    overflow='visible'
                )
            )
            for img_widget, input_widget in zip(image_widgets, self.rating_inputs)
        ]

        # Build the flexbox container
        grid_box = Box(children=image_input_pairs, layout=box_layout)

        controls = HBox([self.button_evolve, self.button_randomize])

        # Display widgets for this generation
        clear_output(wait=True)
        display(VBox([
            self.info_label,
            HTML('<b>Rate each image with emoji:</b>'),
            grid_box,
            controls
        ]))

    def on_evolve_clicked(self, _):
        # Collect ternary ratings from toggle buttons (0, 1, or 2)
        ratings = [int(widget.value) for widget in self.rating_inputs]
        self.population, elites = evolve_population(self.population, ratings)
        best = max(ind.fitness for ind in self.population)
        self.history_best.append(best)
        self.generation += 1
        self.update_display()

    def on_randomize_clicked(self, _):
        self.population = init_population()
        self.generation = 0
        self.history_best = []
        self.update_display()


# =======================
# Launch the Interactive EA
# =======================

iea = InteractiveEA()